 uvicorn api.main:app --reload

http://127.0.0.1:8000/docs#/default/predict_predict_post

In [28]:
import numpy as np
import joblib
import json
import os
import random

# ----------------------------------------------------
# SET RANDOMNESS (comment seed for true randomness)
# ----------------------------------------------------
random.seed()        # different each run
# random.seed(42)    # uncomment for reproducible runs

# ----------------------------------------------------
# RESOLVE PROJECT ROOT (NOTEBOOK-SAFE)
# ----------------------------------------------------
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
MODEL_DIR = os.path.join(PROJECT_ROOT, "models")

SCALER_PATH = os.path.join(MODEL_DIR, "scaler.pkl")
AUTOENCODER_PATH = os.path.join(MODEL_DIR, "autoencoder.pkl")
X_VAL_PATH = os.path.join(MODEL_DIR, "X_val_scaled.npy")

# ----------------------------------------------------
# LOAD ARTIFACTS
# ----------------------------------------------------
scaler = joblib.load(SCALER_PATH)
autoencoder = joblib.load(AUTOENCODER_PATH)
X_val = np.load(X_VAL_PATH)

feature_names = list(scaler.feature_names_in_)

# ----------------------------------------------------
# COMPUTE RECONSTRUCTION ERRORS
# ----------------------------------------------------
X_val_pred = autoencoder.predict(X_val)
reconstruction_errors = ((X_val - X_val_pred) ** 2).mean(axis=1)

THRESHOLD = np.percentile(reconstruction_errors, 99.3)

# ----------------------------------------------------
# HELPER → API INPUT
# ----------------------------------------------------
def to_api_input(idx):
    x_scaled = X_val[idx].reshape(1, -1)
    x_raw = scaler.inverse_transform(x_scaled)[0]
    return {
        "features": {
            fname: float(value)
            for fname, value in zip(feature_names, x_raw)
        }
    }

# ----------------------------------------------------
# RANDOMLY SELECT LOW / MEDIUM / HIGH
# ----------------------------------------------------
low_candidates = np.where(reconstruction_errors < THRESHOLD * 0.4)[0]
medium_candidates = np.where(
    (reconstruction_errors >= THRESHOLD * 0.8) &
    (reconstruction_errors < THRESHOLD)
)[0]
high_candidates = np.where(reconstruction_errors > THRESHOLD * 2.0)[0]

low_idx = random.choice(low_candidates)
medium_idx = random.choice(medium_candidates)
high_idx = random.choice(high_candidates)

# ----------------------------------------------------
# PRINT API-READY JSON
# ----------------------------------------------------
print("\n================ LOW RISK =================")
print(json.dumps(to_api_input(low_idx), indent=2))

print("\n=============== MEDIUM RISK ===============")
print(json.dumps(to_api_input(medium_idx), indent=2))

print("\n================ HIGH RISK ================")
print(json.dumps(to_api_input(high_idx), indent=2))



================ LOW RISK =================
{
  "features": {
    "Time": 172482.0,
    "Amount": 14.989999999999995,
    "V1": 0.125846281355986,
    "V2": 0.988664351594994,
    "V3": -0.40691718995358006,
    "V4": -0.52134182551964,
    "V5": 0.824695333188918,
    "V6": -0.851442833498624,
    "V7": 0.999884306900652,
    "V8": -0.101548687521601,
    "V9": -0.011266861427355,
    "V10": -0.83654073846347,
    "V11": -0.699753283299373,
    "V12": -0.262558639058691,
    "V13": -0.293930760669305,
    "V14": -0.954816249566328,
    "V15": -0.209678175557768,
    "V16": 0.334587875828973,
    "V17": 0.398717481304045,
    "V18": -0.190220231578735,
    "V19": -0.24710230133582797,
    "V20": 0.023548035924520996,
    "V21": -0.33534627123967,
    "V22": -0.851040250845144,
    "V23": 0.131911160366041,
    "V24": 0.6201099349406058,
    "V25": -0.432346588383427,
    "V26": 0.109946440750165,
    "V27": 0.216490721201698,
    "V28": 0.084999456098424,
    "home_lat": 52.9099277989